In [1]:
import pyparsing as pp


In [123]:
# comments need to be removed
comment = (pp.cStyleComment | pp.cppStyleComment).suppress()

identifier = pp.Word(pp.alphas+'_~', pp.alphanums+'_')
persistency = pp.Keyword('static')
volatility = pp.Keyword('const') | pp.Keyword('volatile')

type_identifier = pp.Forward()
type_identifier <<= pp.Group( pp.Group(pp.Optional(persistency) + pp.Optional(volatility))('keywords') \
                + pp.Group(
                  (pp.Optional(pp.Keyword('unsigned') | pp.Keyword('signed')) \
                        + (pp.Keyword('double') | pp.Keyword('int') | pp.Keyword('float') | \
                           pp.Keyword('char') | pp.Keyword('void'))).setParseAction( lambda tokens: ' '.join(tokens) ) \
                | (identifier + pp.ZeroOrMore(pp.Literal('::') + identifier)).setParseAction( lambda tokens: ''.join(tokens)) \
                + pp.Optional( pp.Literal('<') + type_identifier + pp.Literal('>') ) \
                + pp.ZeroOrMore( pp.Literal('*') | pp.Literal('&') ) )('tp') )

parameter_list = pp.Literal('(').suppress() \
                   + pp.Group(
                         pp.Optional(pp.Group(type_identifier + identifier) \
                       + pp.ZeroOrMore(pp.Literal(',').suppress() + pp.Group(type_identifier + identifier)))
                   ) \
               + pp.Literal(')').suppress()

decl = type_identifier + identifier + pp.ZeroOrMore(pp.Literal(',').suppress() + identifier)
fun_decl = pp.Optional(pp.Keyword('virtual'))('keywords') \
         + pp.Optional(type_identifier)('ret_type') + identifier('name') \
         + parameter_list('parameters') \
         + pp.Optional(pp.Keyword('const'))

visibility = pp.Keyword('private') | pp.Keyword('public') | pp.Keyword('protected')

class_decl = pp.Keyword('class').suppress() + identifier
class_def  = pp.Optional(visibility) + class_decl + \
             pp.Optional(pp.Group(pp.Literal(':').suppress() + pp.Optional(visibility) + identifier))


In [53]:
type_definition = pp.Keyword('typedef') + type_identifier + identifier

In [3]:
pp.nestedExpr?

In [10]:
import os
from os.path import join, getsize


In [79]:
def print_args(function):
    def wrapper(*args, **kwargs):
        print('arguments: %s, %s' % (str(args), str(kwargs)))
        return function(*args, **kwargs)
    return wrapper


In [135]:
source_code = '\n'.join(
    open('/home/farad/Programmieren/flightgear-flightgear/src/Input/FGEventInput.hxx')
)

source_code = comment.transformString(source_code)

functions = fun_decl.searchString(source_code)
decls = decl.searchString(source_code)

def type_identifier_to_str(t):
    return str(t[0].keywords[0])

def function_to_str(f):
    return f.keywords + ' ' + type_identifier_to_str(f.ret_type) + ' ' + f.name

for i,f in enumerate(functions):
    f_str = ""
    for kw in f.ret_type[0][0]:
        f_str += str(kw) + ' '
    if len(f.ret_type[0][1]) > 0:
        f_str += ''.join(f.ret_type[0][1]) + ' '
    f_str += f.name
    print(('%d: ' % i) + f_str)

for dcl in decls:
    print(dcl)

0: bool Test
1: double GetValue
2: irtual ~FGInputEvent
3: void fire
4: std::string GetName
5: std::string GetDescription
6: void update
7: static FGInputEvent* NewObject
8: void fire
9: void fire
10: void SetMaxRange
11: void SetMinRange
12: void SetRange
13: void fire
14: void fire
15: void fire
16: irtual ~FGInputDevice
17: void Open
18: void Close
19: void Send
20: har* TranslateEventName
21: void SetName
22: std::string& GetName
23: void HandleEvent
24: void AddHandledEvent
25: void Configure
26: void update
27: bool GetDebugEvents
28: bool GetGrab
29: const std::string& GetNasalModule
30: irtual ~FGEventInput
31: void init
32: void postinit
33: void update
34: unsigned AddDevice
35: void RemoveDevice
[[[], ['ifndef']], '__FGEVENTINPUT_HXX']
[[[], ['define']], '__FGEVENTINPUT_HXX']
[[[], ['struct']], 'FGEventData']
[[[], ['double']], 'aValue', 'double']
[[[], ['int']], 'aModifiers']
[[[], ['int']], 'modifiers']
[[[], ['double']], 'value']
[[[], ['double']], 'dt']
[[[], ['class']],

In [24]:
typedefs = type_definition.searchString(
    '\n'.join(
        open('/home/farad/Programmieren/flightgear-flightgear/src/Input/FGCommonInput.hxx')
    )
)

for td in typedefs:
    print(td)

['typedef', ['std', '::', 'vector', '<', ['SGSharedPtr', '<', ['SGBinding'], '>'], '>'], 'binding_list_t']


In [68]:
classes = set()
derivations = []

for root, dirs, files in os.walk('/home/farad/Programmieren/flightgear-flightgear/src/'):
    for f in filter((lambda f: f.endswith('.hxx')), files):
        p = join(root, f)
        #print('%s:' % p)
        # file content with comments removed
        file_content = comment.transformString('\n'.join(open(p)))
        for cd in class_def.searchString(file_content):
            classes.add(cd[0])
            if len(cd) > 1:
                base_classes = { e[-1] for e in cd[1:] }
                classes.update(base_classes)
                derivations.extend([ (cd[0], bc) for bc in base_classes ])


In [ ]:
derivations

In [70]:
len(classes)

707

In [55]:
import networkx as nx

In [71]:
gr = nx.DiGraph()
gr.add_nodes_from(classes)
gr.add_edges_from(derivations)

In [95]:
nx.write_gexf(gr, '/home/farad/test.gexf')

In [73]:
def write_graph_csv(output_file, graph):
    with open(output_file, 'w') as ostream:
        ostream.write('Source,Target\n')
        for edge in graph.edges_iter():
            ostream.write('%s,%s\n' % edge)


In [74]:
write_graph_csv('/home/farad/test.csv', gr)